# cDLinear; F = 10; T = 1000


In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from synthetic import simulate_lorenz_96
from cdlinear import cDLinear, train_model_ista

In [ ]:
# For GPU acceleration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Simulate data
X_np, GC = simulate_lorenz_96(p=20, F=10, T=1000)
X = torch.tensor(X_np[np.newaxis], dtype=torch.float32, device=device)

In [7]:
# Plot data
fig, axarr = plt.subplots(1, 2, figsize=(16, 5))
axarr[0].plot(X_np)
axarr[0].set_xlabel('T')
axarr[0].set_title('Entire time series')
axarr[1].plot(X_np[:50])
axarr[1].set_xlabel('T')
axarr[1].set_title('First 50 time points')
plt.tight_layout()

plt.show()

In [ ]:
  #Set up model
  cdlinear = cDLinear(X.shape[-1], hidden=100, lag=5).to(device=device)

  # Train with ISTA
  train_loss_list = train_model_ista(
    cdlinear, X, lam=26, lam_ridge=1e-5, lr=5e-5, penalty='H', max_iter=50000,
    check_every=100)

  # Loss function plot
  plt.figure(figsize=(8, 5))
  train_loss_np = [loss.cpu().detach().numpy() for loss in train_loss_list]
  plt.plot(50 * np.arange(len(train_loss_np)), train_loss_np)
  plt.title('cDLinear training')
  plt.ylabel('Loss')
  plt.xlabel('Training steps')
  plt.tight_layout()
  plt.show()

  # Verify learned Granger causality
  GC_est = cdlinear.GC().cpu().data.numpy()

  # Make figures for Granger causality matrices
  fig, axarr = plt.subplots(1, 2, figsize=(16, 5))
  axarr[0].imshow(GC, cmap='Blues')
  axarr[0].set_title('GC actual')
  axarr[0].set_ylabel('Affected series')
  axarr[0].set_xlabel('Causal series')
  axarr[0].set_xticks([])
  axarr[0].set_yticks([])

  axarr[1].imshow(GC_est, cmap='Blues', vmin=0, vmax=1, extent=(0, len(GC_est), len(GC_est), 0))
  axarr[1].set_title('GC estimated')
  axarr[1].set_ylabel('Affected series')
  axarr[1].set_xlabel('Causal series')
  axarr[1].set_xticks([])
  axarr[1].set_yticks([])

  # Mark disagreements
  for i in range(len(GC_est)):
    for j in range(len(GC_est)):
        if GC[i, j] != GC_est[i, j]:
            rect = plt.Rectangle((j, i-0.05), 1, 1, facecolor='none', edgecolor='red', linewidth=1)
            axarr[1].add_patch(rect)

  plt.show()

----------Iter = 100----------
Loss = 820.152283
nonsmooth = 690.540466
smooth = 129.611786
Variable usage = 100.00%
----------Iter = 200----------
Loss = 772.074646
nonsmooth = 651.128052
smooth = 120.946556
Variable usage = 100.00%
----------Iter = 300----------
Loss = 729.241882
nonsmooth = 611.656006
smooth = 117.585915
Variable usage = 100.00%
----------Iter = 400----------
Loss = 688.196350
nonsmooth = 572.117065
smooth = 116.079285
Variable usage = 100.00%
----------Iter = 500----------
Loss = 647.858215
nonsmooth = 532.546753
smooth = 115.311485
Variable usage = 100.00%
----------Iter = 600----------
Loss = 607.848206
nonsmooth = 492.976166
smooth = 114.872025
Variable usage = 100.00%
----------Iter = 700----------
Loss = 568.025818
nonsmooth = 453.429535
smooth = 114.596245
Variable usage = 100.00%
----------Iter = 800----------
Loss = 528.339478
nonsmooth = 413.926971
smooth = 114.412529
Variable usage = 100.00%
----------Iter = 900----------
Loss = 488.776764
nonsmooth = 374